<a href="https://colab.research.google.com/github/rshekhar0/mscb1/blob/main/Create_a_blockchain%2C_a_genesis_block%2C_and_execute_it.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import Crypto  # Importing the Crypto module for cryptographic functions
import binascii  # Importing the binascii module for binary-to-ASCII conversion
import datetime  # Importing the datetime module for handling date and time
import collections  # Importing the collections module for OrderedDict

from Crypto.PublicKey import RSA  # Importing the RSA class from the Crypto.PublicKey module for RSA key generation
from Crypto.Hash import SHA  # Importing the SHA hashing algorithm from the Crypto.Hash module
from Crypto.Signature import PKCS1_v1_5  # Importing the PKCS1_v1_5 class from the Crypto.Signature module for RSA signature

class Client:
    """
    Class representing a client with RSA key pair
    """
    def __init__(self):
        # Creating random number for key
        random = Crypto.Random.new().read
        # Creating new public key and private key
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        # Converting the public key to DER format, then converting it to ASCII hex representation
        return binascii.hexlify(self._public_key.exportKey(format="DER")).decode("ascii")


class Transaction:
    """
    Class representing a transaction between two clients
    """
    def __init__(self, sender, receiver, value):
        self.sender = sender
        self.receiver = receiver
        self.value = value
        self.time = datetime.datetime.now()

    def to_dict(self):
        # Create an ordered dictionary with transaction details
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        return collections.OrderedDict(
            {
                "sender": identity,
                "receiver": self.receiver,
                "value": self.value,
                "time": self.time,
            }
        )

    def sign_transaction(self):
        # Sign the transaction using the sender's private key
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode("utf8"))
        return binascii.hexlify(signer.sign(h)).decode("ascii")


class Block:
    """
    Class representing a block in the blockchain
    """
    def __init__(self):
        self.verified_transactions = []  # List of verified transactions
        self.previous_block_hash = ""  # Hash of the previous block
        self.Nonce = ""  # Nonce value for proof of work

    @staticmethod
    def display_transaction(transaction):
        # Display details of a transaction
        dict_data = transaction.to_dict()
        print("Sender: " + dict_data["sender"])
        print("Receiver: " + dict_data["receiver"])
        print("Value: " + str(dict_data["value"]))
        print("Time: " + str(dict_data["time"]))

# Create a client
Rahul = Client()

# Create a genesis transaction
t0 = Transaction("Genesis", Rahul.identity, 500.0)

# Create a genesis block
block0 = Block()
block0.previous_block_hash = None
block0.verified_transactions.append(t0)
TPCoins = [block0]  # List to store blocks

def dump_blockchain(blocks):
    """
    Function to dump the blockchain
    """
    print("Number of blocks in chain: " + str(len(blocks)))
    for block_index, block in enumerate(blocks):
        print("Block #" + str(block_index))
        for transaction in block.verified_transactions:
            Block.display_transaction(transaction)
            print("-" * 20)
        print("=" * 30)

# Dump the blockchain
dump_blockchain(TPCoins)


Number of blocks in chain: 1
Block #0
Sender: Genesis
Receiver: 30819f300d06092a864886f70d010101050003818d0030818902818100c9cc92454c1d9117ce648eb96dc4a117d986040ebd2a696b4ba902f2eb8547e0446d66d7db536b98f59834f319c80bbee9adb6f53ca78dab5b000bd35c4e0b77f1ccb28a13f8ab4c728f4efcb416c95357aa4e683314a80e6418cf0870e4e4183b292226ef44b6d14e15069662ac004cfa56488613150b490976938476635cc30203010001
Value: 500.0
Time: 2024-05-10 01:41:54.819537
--------------------
